# Part I: Scraping MTG Card Decks


In [2]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import re
import time
import pandas as pd
import mtgutilities as mtgu
import json
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
import requests

In [3]:
with open('config.json', 'r') as config_file:
    contents = json.loads(config_file.read())

### Scraping deck URLs and metadata

In [23]:
URL = 'https://edhrec.com/'
COLORS = ['w', 'u', 'b', 'g', 'r', 'colorless', 'wu', 'ub', 'br', 'rg', 'gw', 'wb', 'ur', 'bg', 'rw', 'gu', 'wub', 'ubr',
    'brg', 'rgw', 'gwu', 'wbg', 'uwr', 'bgu', 'rwb', 'gur', 'wubr', 'ubrg', 'brgw', 'rgwu', 'gwub', 'wubrg']
colors_urls = [f"{URL}commanders/{color}" for color in COLORS]

commander_colors = {}
commanders = {}
decks = {}
for commander_color_url in colors_urls[17:]:
    commander_list = []

    # Enter each commander color site, click Text View
    chrome_options = webdriver.ChromeOptions()
    opts = Options()
    opts.add_argument("--incognito")
    driver = webdriver.Chrome(contents["chrome-driver-location"], options=opts)
    driver.get(commander_color_url)
    mtgu.click_element_in_href(driver, 'Text View')
    time.sleep(2)

    # Clicking Read More to reveal all commanders
    elements = driver.find_elements(By.CSS_SELECTOR, "button.btn.btn-primary")
    while len(elements)>1:
        try:
            element = driver.find_elements(By.CSS_SELECTOR, "button.btn.btn-primary")[1]
            element.click()
            time.sleep(2)
        except IndexError:
            break
    
    # Scraping Commander names
    a = driver.page_source
    soup = bs(a, features="html.parser")
    for c in soup.find_all('a', href=True):
        if f"/commanders/" in c["href"]:
            commander_list.append(c['href'][1:])
    
    decks = {}
    # Scraping all deck URLs for each commander
    for commander in commander_list:
        chrome_options = webdriver.ChromeOptions()
        opts = Options()
        opts.add_argument("--incognito")
        driver = webdriver.Chrome(contents["chrome-driver-location"], options=opts)
        driver.get(f"{URL}decks/{commander[11:]}")
        a = driver.page_source
        soup_commander = bs(a, features="html.parser")
        
        while True:
            a = driver.page_source
            soup_commander = bs(a, features="html.parser")
            for a in soup_commander.find_all('a', href=True):
                if '/deckpreview/' in a["href"]:
                    deck_id = a["href"]
                    deck_data = {}
                    for b in soup_commander.find_all("td"):
                        if deck_id in str(b):
                            deck_data['price'] = b.findNextSibling().text
                            deck_data['tribe'] = b.findNextSibling().findNextSibling().text
                            deck_data['theme'] = b.findNextSibling().findNextSibling().findNextSibling().text
                            decks[deck_id] = deck_data
            
            if mtgu.check_exists_by_xpath(driver, "//a[contains(text(), 'Next')]"):
                element = driver.find_element(By.XPATH, "//a[contains(text(), 'Next')]")
                driver.execute_script('arguments[0].scrollIntoView();', element)
                driver.execute_script('window.scrollBy(0, -200);')
                element.click()
            else:
                break
        commanders[commander] = decks
    driver.close()
    commander_colors[commander_color_url] = commanders

WebDriverException: Message: unknown error: cannot create temp dir for user data dir


In [19]:
import gzip
import json
with gzip.open('commanders_deckurls2.json.gz', 'wt', encoding="ascii") as zipfile:
       json.dump(commander_colors, zipfile)

In [8]:
with open('my_dict.json') as f:
    my_dict = json.load(f)

In [21]:
colors_urls

['https://edhrec.com/commanders/w',
 'https://edhrec.com/commanders/u',
 'https://edhrec.com/commanders/b',
 'https://edhrec.com/commanders/g',
 'https://edhrec.com/commanders/r',
 'https://edhrec.com/commanders/colorless',
 'https://edhrec.com/commanders/wu',
 'https://edhrec.com/commanders/ub',
 'https://edhrec.com/commanders/br',
 'https://edhrec.com/commanders/rg',
 'https://edhrec.com/commanders/gw',
 'https://edhrec.com/commanders/wb',
 'https://edhrec.com/commanders/ur',
 'https://edhrec.com/commanders/bg',
 'https://edhrec.com/commanders/rw',
 'https://edhrec.com/commanders/gu',
 'https://edhrec.com/commanders/wub',
 'https://edhrec.com/commanders/ubrbrg',
 'https://edhrec.com/commanders/rgw',
 'https://edhrec.com/commanders/gwu',
 'https://edhrec.com/commanders/wbg',
 'https://edhrec.com/commanders/uwr',
 'https://edhrec.com/commanders/bgu',
 'https://edhrec.com/commanders/rwb',
 'https://edhrec.com/commanders/gur',
 'https://edhrec.com/commanders/wubr',
 'https://edhrec.com/c